<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

# Ditto - avec pandas

In [ ]:
import pandas as pd

In [ ]:
pd.options.display.max_rows = 8

# digression

À titre expérimental, nous importons un notebook comme un module python

In [ ]:
# un loader personnalisé pour charger un notebook comme un module
from notebookloader import NotebookFinder

In [ ]:
import sys
sys.meta_path.append(NotebookFinder())

In [ ]:
# avec ce bidule on peut directement charger un notebook comme un module
from airports import distance_degrees

# airports

In [ ]:
airports_url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat"

In [ ]:
airports = pd.read_csv(airports_url, header=None)

In [ ]:
# si on n'en dit pas plus on obtient les données mais les noms laissent à désirer
airports

In [ ]:
# on peut donner des noms aux colonnes
airports.columns = ["airport_id", "name", "city", "country", "iata", "icao", "latitude", "longitude", "altitude", "timezone", "dst", "tz", "airport", "source"]

In [ ]:
# ne voir que le début
airports.head()

In [ ]:
# un échantillon
airport1 = airports.iloc[0]; airport1

In [ ]:
# pour information
type(airport1['airport_id'])

In [ ]:
airports.loc?

# airlines

pareil...

In [ ]:
airlines_url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat"

In [ ]:
airlines = pd.read_csv(airlines_url, header=None)

In [ ]:
airlines.columns = ["airline_id", "name", "alias", "iata", "icao", "callsign", "country", "active"]

In [ ]:
airlines.head(20)

In [ ]:
# un échantillon
airline1 = airlines.iloc[2]; airline1

# routes

et encore une fois

In [ ]:
routes_url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat"

In [ ]:
routes = pd.read_csv(routes_url, header=None)

In [ ]:
routes.columns = ["airline", "airline_id",
                  "src_airport", "src_airport_id", "dst_airport", "dst_airport_id",
                  "codeshare", "stops", "equipment"]

In [ ]:
routes.tail(20)

### un peu de nettoyage

**attention** certaines ces lignes contiennent un `airline_id` - entre autres - non renseigné

comme avec *numpy* on peut faire des opérations de comparaison, pour ne retenir par exemple ici que les lignes qui utilisables

In [ ]:
valid_series = (routes['airline_id'] == r'\N')

In [ ]:
# cette expression renvoie une Series (colonne)
valid_series

In [ ]:
type(valid_series)

In [ ]:
# comme la colonne est faite de booléens, 
# on peut s'en servir pour indexer dans la table
unusable_routes = routes[(routes['airline_id'] == r'\N')]
unusable_routes.iloc[0]

In [ ]:
# le nombre d'entrées inutilisables 
# par rapport à airline_id
len(unusable_routes)

In [ ]:
# le nombre total de routes exposées dans l'URL
nb_total_routes = len(routes)

In [ ]:
# pour filtrer, c'est à dire ne garder 
# que les lignes qui vérifient une condition
# ici on étend un peu le critère de validité
routes = routes[(routes['airline_id'] != r'\N') 
              & (routes['src_airport_id'] != r'\N') 
              & (routes['dst_airport_id'] != r'\N')]

In [ ]:
# on prend une colonne, on la traduit en nombre
# car sinon on n'a que des chaines de caractères
routes['airline_id'] = pd.to_numeric(routes['airline_id'])

In [ ]:
# maintenant on a bien un entier
route1 = routes.iloc[0]
type(route1['airline_id'])

In [ ]:
# en fait, plusieurs colonnes sont dans ce cas
# et ont besoin d'être traduites en nombres
for column in ('airline_id', 'src_airport_id', 'dst_airport_id'):
    routes[column] = pd.to_numeric(routes[column])

In [ ]:
# sauf qu'à ce stade on a encore un souci de nettoyage
# car pour certaines routes, un des deux aéroports est inconnu
routes.iloc[170].dst_airport_id

In [ ]:
airports[airports['airport_id'] == 7163]

In [ ]:
# il faut donc nettoyer davantage
# XXX here...

### distance

In [ ]:
# rappel: on a importé distance_degrees du notebook 'airports'

# ici route est une ligne dans routes
def distance_route(route):
    src_airport = airports[airports['airport_id'] == route['src_airport_id']].iloc[0]
    dst_airport = airports[airports['airport_id'] == route['dst_airport_id']].iloc[0]
    return distance_degrees(
        src_airport['latitude'], src_airport['longitude'],
        dst_airport['latitude'], dst_airport['longitude'])

In [ ]:
# on peut maintenant appliquer cette fonction à une route 
# c'est à dire une ligne de la table
route0 = routes.iloc[0]
distance_route(route0)

In [ ]:
route0

### un histogramme

pour représenter la répartition des longueurs de routes

In [ ]:
lengths = routes.apply(distance_route, axis=1)